In [1]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

# General imports
import numpy as np

from qiskit.quantum_info import SparsePauliOp

# custom module
from susy_qm import calculate_Hamiltonian

from scipy.optimize import minimize, differential_evolution
from scipy.stats.qmc import Halton

import json
import os

In [54]:
potential = 'DW'
cutoff = 16
shots = None#'1024

In [55]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])
min_eigenvalue = min(eigenvalues.real)

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [56]:
min_eigenvalue

np.float64(0.8915993623272539)

In [57]:
cnot_pool = []
cz_pool = []

for control in range(num_qubits):
        for target in range(num_qubits):
            if control != target:
                cnot_pool.append(qml.CNOT(wires=[control, target]))
                cz_pool.append(qml.CZ(wires=[control, target]))

In [58]:
RY_pool = [qml.RY(0.0, wires=x) for x in range(num_qubits)]
RZ_pool = [qml.RZ(0.0, wires=x) for x in range(num_qubits)]
RX_pool = [qml.RX(0.0, wires=x) for x in range(num_qubits)]
operator_pool = cz_pool + RY_pool + RZ_pool + RX_pool

In [59]:
operator_pool

[CZ(wires=[0, 1]),
 CZ(wires=[0, 2]),
 CZ(wires=[0, 3]),
 CZ(wires=[0, 4]),
 CZ(wires=[1, 0]),
 CZ(wires=[1, 2]),
 CZ(wires=[1, 3]),
 CZ(wires=[1, 4]),
 CZ(wires=[2, 0]),
 CZ(wires=[2, 1]),
 CZ(wires=[2, 3]),
 CZ(wires=[2, 4]),
 CZ(wires=[3, 0]),
 CZ(wires=[3, 1]),
 CZ(wires=[3, 2]),
 CZ(wires=[3, 4]),
 CZ(wires=[4, 0]),
 CZ(wires=[4, 1]),
 CZ(wires=[4, 2]),
 CZ(wires=[4, 3]),
 RY(0.0, wires=[0]),
 RY(0.0, wires=[1]),
 RY(0.0, wires=[2]),
 RY(0.0, wires=[3]),
 RY(0.0, wires=[4]),
 RZ(0.0, wires=[0]),
 RZ(0.0, wires=[1]),
 RZ(0.0, wires=[2]),
 RZ(0.0, wires=[3]),
 RZ(0.0, wires=[4]),
 RX(0.0, wires=[0]),
 RX(0.0, wires=[1]),
 RX(0.0, wires=[2]),
 RX(0.0, wires=[3]),
 RX(0.0, wires=[4])]

In [60]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)

@qml.qnode(dev)
def circuit(params, trial_op, op_list):

    if len(op_list) > 0:
        for o, p, w, _ in op_list:
            if (o == qml.CNOT) | (o == qml.CZ):
                o(wires=w)
            else:
                o(p, wires=w)

    op = type(trial_op)

    if (type(trial_op) == qml.CNOT) | (type(trial_op) == qml.CZ):
        op(wires=trial_op.wires)
    else:
        op(params, wires=trial_op.wires)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [61]:
def cost_function(params, trial_op, op_list):
    
    params = pnp.tensor(params, requires_grad=True)
    energy = circuit(params, trial_op, op_list)

    return energy

In [62]:
num_steps = 10

x0 = np.random.rand()
bounds = [(0, 2 * np.pi) for _ in range(3)]

#variables
max_iter = 1000
strategy = "randtobest1bin"
tol = 1e-2
abs_tol = 1e-2
popsize = 20

# Generate Halton sequence
num_dimensions = np.prod(3)
num_samples = popsize
halton_sampler = Halton(d=num_dimensions)
halton_samples = halton_sampler.random(n=num_samples)
scaled_samples = 2 * np.pi * halton_samples

op_list = []

for i in range(num_steps):

    print(f"step: {i}")

    energies = []
    e_params = []

    for trial_op in operator_pool:

        res = minimize(
                cost_function,
                x0,
                args=(trial_op, op_list),
                method= "COBYLA",
                options= {'maxiter':1000, 'tol': 1e-6}
            )
        '''
        res = differential_evolution(cost_function,
                                    bounds,
                                    args=(trial_op, op_list),
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=abs_tol,
                                    strategy=strategy,
                                    popsize=popsize,
                                    init=scaled_samples
                                    )
        '''
        energies.append(res.fun)
        e_params.append(res.x)

    min_arg = np.argmin(energies)
    min_energy = energies[min_arg]
    print(f"Min energy: {min_energy}")

    min_op = type(operator_pool[min_arg])
    min_wires = operator_pool[min_arg].wires
    min_params = e_params[min_arg]

    if (i != 0):
        if np.abs(min_energy - op_list[-1][3]) < 1e-4:
            print("Converged")
            break

    if (i == 0):
        op_list.append((min_op, min_params, min_wires, min_energy))

    elif (i != 0) & (min_energy <= op_list[-1][3]):
        op_list.append((min_op, min_params, min_wires, min_energy))

    

step: 0
Min energy: 1.0939110867588582
step: 1
Min energy: 1.0919837656605664
step: 2
Min energy: 1.052433979371373
step: 3
Min energy: 1.04932439201529
step: 4
Min energy: 1.047214352880776
step: 5
Min energy: 1.045867830098561
step: 6
Min energy: 1.0457705409701377
Converged


In [63]:
op_list

[(pennylane.ops.qubit.parametric_ops_single_qubit.RY,
  array([-0.95531832]),
  Wires([4]),
  np.float64(1.0939110867588582)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.RY,
  array([0.04068397]),
  Wires([3]),
  np.float64(1.0919837656605664)),
 (pennylane.ops.op_math.controlled_ops.CZ,
  array([0.92364615]),
  Wires([3, 4]),
  np.float64(1.052433979371373)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.RY,
  array([-0.02957331]),
  Wires([2]),
  np.float64(1.04932439201529)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.RY,
  array([-0.03716637]),
  Wires([4]),
  np.float64(1.047214352880776)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.RY,
  array([0.03449162]),
  Wires([3]),
  np.float64(1.045867830098561))]

In [64]:
last_operator = {}
reduced_op_list = []
num_params = 0

for o, p, w, _ in op_list:

    if w[0] in last_operator.keys():
        if last_operator[w[0]] == o:
            print("Operator already added previously... skipping")
            continue

    if o == qml.CNOT:
        last_operator[w[0]] = o
        last_operator[w[1]] = o
        reduced_op_list.append(("CNOT", w.tolist()))

    elif o == qml.CZ:
        last_operator[w[0]] = o
        last_operator[w[1]] = o
        reduced_op_list.append(("CZ",w.tolist()))

    elif o == qml.RY:
        last_operator[w[0]] = o
        reduced_op_list.append(("RY",w.tolist()))
        num_params+=1

    elif o == qml.RZ:
        last_operator[w[0]] = o
        reduced_op_list.append(("RZ",w.tolist()))
        num_params+=1

    elif o == qml.RX:
        last_operator[w[0]] = o
        reduced_op_list.append(("RX",w.tolist()))
        num_params+=1

In [65]:
reduced_op_list

[('RY', [4]),
 ('RY', [3]),
 ('CZ', [3, 4]),
 ('RY', [2]),
 ('RY', [4]),
 ('RY', [3])]

In [66]:
data = {"potential": potential,
        "cutoff": cutoff,
        "optimizer": "COBYLA",
        "num steps": num_steps,
        "op_list": [str(x) for x in op_list],
        "reduced_op_list": reduced_op_list,
        "num_params": num_params}

In [67]:
data

{'potential': 'DW',
 'cutoff': 16,
 'optimizer': 'COBYLA',
 'num steps': 10,
 'op_list': ["(<class 'pennylane.ops.qubit.parametric_ops_single_qubit.RY'>, array([-0.95531832]), Wires([4]), np.float64(1.0939110867588582))",
  "(<class 'pennylane.ops.qubit.parametric_ops_single_qubit.RY'>, array([0.04068397]), Wires([3]), np.float64(1.0919837656605664))",
  "(<class 'pennylane.ops.op_math.controlled_ops.CZ'>, array([0.92364615]), Wires([3, 4]), np.float64(1.052433979371373))",
  "(<class 'pennylane.ops.qubit.parametric_ops_single_qubit.RY'>, array([-0.02957331]), Wires([2]), np.float64(1.04932439201529))",
  "(<class 'pennylane.ops.qubit.parametric_ops_single_qubit.RY'>, array([-0.03716637]), Wires([4]), np.float64(1.047214352880776))",
  "(<class 'pennylane.ops.qubit.parametric_ops_single_qubit.RY'>, array([0.03449162]), Wires([3]), np.float64(1.045867830098561))"],
 'reduced_op_list': [('RY', [4]),
  ('RY', [3]),
  ('CZ', [3, 4]),
  ('RY', [2]),
  ('RY', [4]),
  ('RY', [3])],
 'num_para

In [72]:
params = np.random.uniform(0, 2 * np.pi, size=num_params)

dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def construct_ansatz(reduced_op_list, params):

    params_index = 0

    for o, w in reduced_op_list:
        if o == "CNOT":
            qml.CNOT(wires=w)
        elif o == "CZ":
            qml.CZ(wires=w)
        elif o == "RY":
            qml.RY(params[params_index], wires=w)
            params_index +=1

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [73]:
print(qml.draw(construct_ansatz, show_matrices=False)(reduced_op_list, params))

0: ────────────────────────┤ ╭<𝓗(M0)>
1: ────────────────────────┤ ├<𝓗(M0)>
2: ──RY(2.76)──────────────┤ ├<𝓗(M0)>
3: ──RY(5.07)─╭●──RY(4.80)─┤ ├<𝓗(M0)>
4: ──RY(2.32)─╰Z──RY(6.18)─┤ ╰<𝓗(M0)>


In [75]:
# Save the variable to a JSON file
base_path = r"C:\Users\johnkerf\Desktop\Quantum-Computing\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\NoRotFiles\\"
os.makedirs(base_path, exist_ok=True)

cd = qml.draw(construct_ansatz, show_matrices=False)(reduced_op_list, params)

path = base_path + "cd_{}_{}.txt".format(potential, cutoff)
with open(path, 'w', encoding='utf-8') as f:
    f.write(cd)

In [76]:
# Save the variable to a JSON file
base_path = r"C:\Users\johnkerf\Desktop\Quantum-Computing\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\NoRotFiles\\"
#base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files\\"
os.makedirs(base_path, exist_ok=True)
path = base_path + "{}_{}.json".format(potential, cutoff)
with open(path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [52]:
reduced_op_list

[('Rot', [5]),
 ('Rot', [4]),
 ('CZ', [4, 5]),
 ('Rot', [5]),
 ('Rot', [3]),
 ('Rot', [4]),
 ('CZ', [3, 4]),
 ('Rot', [3])]

In [254]:
qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=num_qubits)

(1, 5, 3)

In [39]:
last_operator = {}
reduced_op_list = []
num_params = 0

for o, p, w, _ in op_list:

    if (o == qml.CNOT) | (o == qml.CZ):
        last_operator[w[0]] = o
        last_operator[w[1]] = o
        reduced_op_list.append((o,w))

    elif w[0] in last_operator.keys():
        if last_operator[w[0]] == o:
            print("Operator already added previously... skipping")
            continue
        else:
            last_operator[w[0]] = o
            reduced_op_list.append((o,w))
            num_params = num_params + 3
    else:
        last_operator[w[0]] = o
        reduced_op_list.append((o,w))
        num_params = num_params + 3

Operator already added previously... skipping
Operator already added previously... skipping


In [47]:
params = np.random.uniform(0, 2 * np.pi, size=num_params)

dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def construct_ansatz(reduced_op_list, params):

    params_index = 0

    for i, (o, w) in enumerate(reduced_op_list):
        if (o == qml.CNOT) | (o == qml.CZ):
            o(wires=w)
        else:
            num_gate_params = o.num_params
            o(*params[params_index:(params_index + num_gate_params)], wires=w)
            params_index = params_index + num_gate_params

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))